In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from sklearn.model_selection import train_test_split

In [2]:
# 1. Data Preprocessing

# Load the dataset
data = pd.read_csv(r"C:\Users\rohit\Downloads\archive\Reddit_Data.csv")

In [3]:
# Drop rows where 'clean_comment' is NaN
data = data.dropna(subset=['clean_comment'])

In [4]:
# Tokenize and pad sequences
max_words = 10000  # You can adjust this based on your dataset
maxlen = 100  # Maximum length of a comment, you can adjust this based on your dataset

In [5]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['clean_comment'])
sequences = tokenizer.texts_to_sequences(data['clean_comment'])

In [6]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

Found 54719 unique tokens.


In [7]:
data_pad = pad_sequences(sequences, maxlen=maxlen)

In [8]:
labels = np.asarray(data['category'])
print('Shape of data tensor:', data_pad.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (37149, 100)
Shape of label tensor: (37149,)


In [9]:
# Split the data into a training set and a validation set
x_train, x_val, y_train, y_val = train_test_split(data_pad, labels, test_size=0.2)

In [10]:
# 2. Model Building

embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))  # Use 'sigmoid' for binary classification

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 gru (GRU)                   (None, 32)                12864     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 1,012,897
Trainable params: 1,012,897
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [12]:
# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/10
929/929 [==============================] - 55s 57ms/step - loss: -0.6177 - acc: 0.5576 - val_loss: -1.7873 - val_acc: 0.6281
Epoch 2/10
929/929 [==============================] - 54s 58ms/step - loss: -4.2443 - acc: 0.6700 - val_loss: -5.3466 - val_acc: 0.6808
Epoch 3/10
929/929 [==============================] - 55s 59ms/step - loss: -8.6587 - acc: 0.6910 - val_loss: -8.7758 - val_acc: 0.6857
Epoch 4/10
929/929 [==============================] - 54s 58ms/step - loss: -13.4149 - acc: 0.6938 - val_loss: -12.5921 - val_acc: 0.7100
Epoch 5/10
929/929 [==============================] - 54s 58ms/step - loss: -18.0492 - acc: 0.6988 - val_loss: -16.4696 - val_acc: 0.6571
Epoch 6/10
929/929 [==============================] - 54s 58ms/step - loss: -23.4004 - acc: 0.7009 - val_loss: -19.9090 - val_acc: 0.7027
Epoch 7/10
929/929 [==============================] - 54s 58ms/step - loss: -28.5512 - acc: 0.7004 - val_loss: -24.4283 - val_acc: 0.6840
Epoch 8/10
929/929 [====================

In [13]:
# 3. Evaluation

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_val, y_val)
print(f'Test accuracy: {accuracy}')

233/233 [==============================] - 3s 12ms/step - loss: -34.4940 - acc: 0.6918
Test accuracy: 0.6917900443077087
